In [1]:
!pip install /kaggle/input/pretrained-models
!pip install /kaggle/input/efficientnet-pytorch

Processing /kaggle/input/pretrained-models
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=62050 sha256=4cb9cf48dd45c2c5bfb0088495f7757c2916deab8603ce996510f05d02e2bc39
  Stored in directory: /root/.cache/pip/wheels/da/6c/2b/fcb88f8bca720a8b83cfe965f0b53c357d4fdeb499b8b2fb20
Successfully built pretrainedmodels
Processing /kaggle/input/efficientnet-pytorch
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.5.1-cp36-none-any.whl size=15931 sha256=5bc88c1e3d01c6517d16be31d6950c01f894ad8d05c70c759c27d83349e01420
  Stored in directory: /root/.cache/pip/wheels/2e/f1/62/ae1329ee474be852cd9c5725524eac804081351f8a1fd0cf6f
Successfully built efficientnet-pytorch


In [2]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [3]:
DATA_DIR = '/kaggle/input/bengaliai-cv19'
MODEL_DIR = '/kaggle/input/bengali-models'

#DATA_DIR = '/mnt/chicm/data/bengali'
#MODEL_DIR = './models'

In [4]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [5]:
test_df.head()

,row_id,image_id,component
0,Test_0_consonant_diacritic,Test_0,consonant_diacritic
1,Test_0_grapheme_root,Test_0,grapheme_root
2,Test_0_vowel_diacritic,Test_0,vowel_diacritic
3,Test_1_consonant_diacritic,Test_1,consonant_diacritic
4,Test_1_grapheme_root,Test_1,grapheme_root


In [6]:
sample_sub_df.head()

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,0
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,0


In [7]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

HEIGHT = 137
WIDTH = 236

class BengaliDataset(Dataset):
    def __init__(self, df, img_df, test_mode=True):
        self.df = df
        self.img_df = img_df
        self.test_mode = test_mode

    def __getitem__(self, idx):
        img = 255 - self.img_df.iloc[idx].values.reshape(HEIGHT, WIDTH).astype(np.uint8)
        img = np.expand_dims(img, axis=-1)
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
        return img

    def __len__(self):
        if self.df is None:
            return len(self.img_df)
        else:
            return len(self.df)


In [8]:
def get_test_loader(batch_size=4, idx=0):
    #img_dfs = [pd.read_parquet(f'{DATA_DIR}/test_image_data_{i}.parquet') for i in range(4)]
    #img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    img_df = pd.read_parquet(f'{DATA_DIR}/test_image_data_{idx}.parquet').set_index('image_id')

    ds = BengaliDataset(None, img_df, test_mode=True)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=False)
    loader.num = len(ds)
    return loader

# model

In [9]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [10]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained=None)
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
    def logits(self, x):
        x = self.avg_pool(x)
        x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [11]:
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

# predict

In [12]:
def predict(model, test_loader):
    preds0, preds1,preds2 = [], [], []
    with torch.no_grad():
        for x in test_loader:
            x = x.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    
    return preds0, preds1, preds2
            

In [13]:
#!ls /kaggle/input/bengali-models

In [14]:
args = Namespace()
args.backbone = 'densenet201'
args.ckp_name = 'densenet201-cv9796.pth'
args.predict = True

In [15]:
import gc

model = create_model(args)[0].cuda()
model.eval()

preds0, preds1, preds2 = [], [], []

for i in range(4):
    test_loader = get_test_loader(batch_size=128, idx=i)
    p0, p1, p2 = predict(model, test_loader)
    preds0.append(p0)
    preds1.append(p1)
    preds2.append(p2)
    del test_loader
    gc.collect()
    
preds0 = np.concatenate(preds0, 0)
preds1 = np.concatenate(preds1, 0)
preds2 = np.concatenate(preds2, 0)

model file: /kaggle/input/bengali-models/densenet201-cv9796.pth, exist: True
loading /kaggle/input/bengali-models/densenet201-cv9796.pth...


In [16]:
row_id = []
target = []
for i in tqdm(range(len(preds0))):
    row_id += [f'Test_{i}_grapheme_root', f'Test_{i}_vowel_diacritic',
               f'Test_{i}_consonant_diacritic']
    target += [preds0[i], preds1[i], preds2[i]]
submission_df = pd.DataFrame({'row_id': row_id, 'target': target})
#submission_df.to_csv('submission.csv', index=False)
submission_df.head()

100%|██████████| 12/12 [00:00<00:00, 18409.53it/s]


,row_id,target
0,Test_0_grapheme_root,3
1,Test_0_vowel_diacritic,0
2,Test_0_consonant_diacritic,0
3,Test_1_grapheme_root,93
4,Test_1_vowel_diacritic,2


In [17]:
submission_df.to_csv('submission.csv', index=False)

In [18]:
submission_df.shape

(36, 2)